## Classification (b)

In [1]:
from neuralnet import *
from random import shuffle
import numpy as np

# Load the configuration.

x_train, y_train, stats = load_data(path="./data",stats = None, mode="train")
# Get 10 examples, 1 from each category.
X_sub = []
y_sub = []
for k in range(10):
    indices = y_train[:,k] == 1
    X_sub.append(x_train[indices][0])
    y_sub.append(y_train[indices][0])
print(len(X_sub), len(y_sub))

# Load model para
config_prob_b = {}
config_prob_b['layer_specs'] = [3072, 10, 10]
config_prob_b['activation'] = 'ReLU'
config_prob_b['learning_rate'] = 0.15 
config_prob_b['batch_size'] = 128 
config_prob_b['epochs'] = 100  
config_prob_b['early_stop'] = True 
config_prob_b['early_stop_epoch'] = 5  
config_prob_b['L2_penalty'] = 0  
config_prob_b['momentum'] = True  
config_prob_b['momentum_gamma'] = 0.9  

model_b = Neuralnetwork(config_prob_b)
model_b.forward(np.array(X_sub), np.array(y_sub))
model_b.backward()




(50000, 32, 32, 3)
inp: (50000, 32, 32, 3)
10 10


In [2]:
# d_b d_w comparison

## Part (b) Estimation of bias weight and weight
def Num_Est_b(model, layer, eps, input_idx, output_idx):
    layer.b[0][output_idx] += eps # E(w+e)
    loss_1 = model.forward(np.array(X_sub), np.array(y_sub))[1]
    layer.b[0][output_idx] -= 2*eps # E(w-e)
    loss_2 = model.forward(np.array(X_sub), np.array(y_sub))[1]
    layer.b[0][output_idx] += eps # back to normal
    return abs(loss_1 - loss_2) / (2 * eps) # Numerical estimation for dEdW

def Num_Est_w(model, layer, eps, input_idx, output_idx):
    layer.w[input_idx][output_idx] += eps # E(w+e)
    loss_1 = model.forward(np.array(X_sub), np.array(y_sub))[1]
    layer.w[input_idx][output_idx] -= 2*eps # E(w-e)
    loss_2 = model.forward(np.array(X_sub), np.array(y_sub))[1]
    layer.w[input_idx][output_idx] += eps # back to normal
    return abs(loss_1 - loss_2) / (2 * eps) # Numerical estimation for dEdW

epss = [1e-2, 1e-4] # difference expectation to 1e-4
db = []
dw = []
db_est = []
dw_est = []

for eps in epss:
    db = []
    dw = []
    db_est = []
    dw_est = []
    for layer in model_b.layers:
        if isinstance(layer, Layer):
            db_est.append(Num_Est_b(model_b, layer, eps, 0, 1))
            db.append(layer.d_b[1]) 
            dw_est.append([Num_Est_w(model_b, layer, eps, 0, 0), Num_Est_w(model_b,layer, eps, 0, 2)])
            dw.append([layer.d_w[0][0] ,layer.d_w[0][2]])   
    print('(dw - dw_est): {}'.format([(np.array(dw)-np.array(dw_est))]))
    print('(db - db_est): {}'.format([(np.array(db)-np.array(db_est))]))
    print('(db_est): {}'.format([(np.array(db_est))]))
    print('(db): {}'.format([(np.array(db))]))
    


(dw - dw_est): [array([[ 0.19399662,  0.23052656],
       [-3.60210521,  2.31543866]])]
(db - db_est): [array([-0.21688454, -0.3       ])]
(db_est): [array([0., 0.])]
(db): [array([-0.21688454, -0.3       ])]
(dw - dw_est): [array([[ 0.19399662,  0.23052656],
       [-3.60210521,  2.31543866]])]
(db - db_est): [array([-0.21688454, -0.3       ])]
(db_est): [array([0., 0.])]
(db): [array([-0.21688454, -0.3       ])]
